<a href="https://colab.research.google.com/github/mugambiian/C-coding/blob/main/Football_Score_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Football Matches Prediction Model

#Question formulation

We will create a model that attempts to predict the outcome of footbal matches between two teams : Team 1 and Team 2 based on five features as below;
1. Home team
2. Away team
3. The type of tournament
4. The rank of the home team
5. The rank of the away team

We shall use regression 

1. Polynomial regression
We shall attempt to predict how many goals the home team scores

Logistic regression

We shall attempt to if the home team wins, looses or draws the game


# Metrics of success

We shall use the Root Mean Squared Error (RMSE) to measure the performace of the model. 

We shall measure the accuracy of the logistic regression model against the accuracy score.



#1.Loading and exploring  the dataset

In [2]:
# we import the libraries we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, confusion_matrix
import datetime as dt


In [3]:
# We then load our first dataset

Results = pd.read_csv("/content/results.csv")
Results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True


In [4]:
# We then load our first dataset

Results = pd.read_csv("/content/results.csv")
Results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True


The object data type will beed to be changed to something more appropriate for manipulation

In [5]:
#we load our second dataset
Rankings = pd.read_csv('/content/fifa_ranking.csv')
Rankings

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07


In [6]:
#  We look at the data types for the various columns
Rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rank                     57793 non-null  int64  
 1   country_full             57793 non-null  object 
 2   country_abrv             57793 non-null  object 
 3   total_points             57793 non-null  float64
 4   previous_points          57793 non-null  int64  
 5   rank_change              57793 non-null  int64  
 6   cur_year_avg             57793 non-null  float64
 7   cur_year_avg_weighted    57793 non-null  float64
 8   last_year_avg            57793 non-null  float64
 9   last_year_avg_weighted   57793 non-null  float64
 10  two_year_ago_avg         57793 non-null  float64
 11  two_year_ago_weighted    57793 non-null  float64
 12  three_year_ago_avg       57793 non-null  float64
 13  three_year_ago_weighted  57793 non-null  float64
 14  confederation         

As before the object data type will need to be edited to a more suitable type for manipulation



# 2.Data cleanup

In [7]:
# We take a closer look at our dataset

Results.describe()

,home_score,away_score
count,40839.000000,40839.000000
mean,1.745709,1.188105
std,1.749145,1.405120
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [8]:
# we check for null values

Results.isnull().sum()

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

We don't have any null values


In [9]:
#We check for duplicated values
Results.duplicated().any()

False

We don't have any duplicates as well



In [10]:
# We rename the columns for consistency
Results = Results.rename(columns = {
    'date' : 'Date',
    'home_team ' : 'Home_team',
    'home_score' : 'Home_Score',
    'away_score' : 'Away_Score',
    'tournament' : 'Tournament',
    'city' : 'City',
    'country' : 'Country',
    'neutral' : 'Neutral'

})
Results

,Date,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True


In [11]:
#we take a closer look at our second dataset
Rankings.describe()


,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted
count,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000
mean,101.628086,122.068637,332.302926,-0.009897,61.798602,61.798602,61.004602,30.502377,59.777462,17.933277,59.173916,11.834811
std,58.618424,260.426863,302.872948,5.804309,138.014883,138.014883,137.688204,68.844143,136.296079,40.888849,135.533343,27.106675
min,1.000000,0.000000,0.000000,-72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,56.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,101.000000,0.000000,272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,152.000000,92.790000,525.000000,1.000000,32.250000,32.250000,26.660000,13.330000,21.500000,6.450000,21.250000,4.250000
max,209.000000,1775.030000,1920.000000,92.000000,1158.660000,1158.660000,1169.570000,584.790000,1159.710000,347.910000,1200.770000,240.150000


In [12]:
# we check for null values

Results.isnull().sum()

Date          0
home_team     0
away_team     0
Home_Score    0
Away_Score    0
Tournament    0
City          0
Country       0
Neutral       0
dtype: int64

  We don't have any null values
  

In [13]:
#We rename the columns for consistency
Rankings = Rankings.rename(columns = {
    'country_full' : 'Country_Full',
    'country_abrv' : 'Country_Abrv',
    'total_points' : 'Total_Points',
    'rank_change' : 'Rank_Change',
    'cur_year_avg' : 'Cur_year_Average',
    'cur_year_avg_weighted' : 'Cur_Year_Avg_Weighted',
    'last_year_avg' : 'Last_Year_Avg',
    'last_year_avg_weighted' : 'Last_Year_Avg_Weighted',
    'two_year_ago_avg' : 'Level_of_Education',
    'two_year_ago_weighted' : 'Two_Year_Ago_Weighted',
    'three_year_ago_avg' : 'Three_Year_Ago_Avg',
    'three_year_ago_weighted' : 'Three_Year_Ago_Weighted',
     'confederation' : 'Confederation',
    'rank_date' : 'Date'

})
Rankings

,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,Confederation,Date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07


In [29]:
Rankings.Date = Rankings.Date.apply(lambda x: x.replace('/', '-'))
Results.Date = Results.Date.apply(lambda x: x.replace('/', '-'))


ValueError: ignored

In [25]:
#We change our date to date time 
Rankings.Date = pd.to_datetime(Rankings.Date)
Results.Date = pd.to_datetime(Results.Date)

In [26]:
#We need to split the date into year and month
Rankings['year'] = Rankings.Date.dt.year
Results['year'] = Results.Date.dt.year

Rankings['month'] = Rankings.Date.dt.month
Results['month'] = Rankings.Date.dt.month




In [17]:
#We confirm that the date columns have changed to datetime objects
Rankings.info()
Results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   rank                     57793 non-null  int64         
 1   Country_Full             57793 non-null  object        
 2   Country_Abrv             57793 non-null  object        
 3   Total_Points             57793 non-null  float64       
 4   previous_points          57793 non-null  int64         
 5   Rank_Change              57793 non-null  int64         
 6   Cur_year_Average         57793 non-null  float64       
 7   Cur_Year_Avg_Weighted    57793 non-null  float64       
 8   Last_Year_Avg            57793 non-null  float64       
 9   Last_Year_Avg_Weighted   57793 non-null  float64       
 10  Level_of_Education       57793 non-null  float64       
 11  Two_Year_Ago_Weighted    57793 non-null  float64       
 12  Three_Year_Ago_Avg       57793 n

In [35]:
#We combine the two datasets to begin our analysis
Combined_Data = pd.merge(Results, Rankings, how = 'left', left_on = ['year', 'month'], right_on = ['year', 'month'])
Combined_Data

,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,Confederation,Date_y
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4478938,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478939,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478940,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478941,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [19]:
#We preview our datasets
Combined_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4478943 entries, 0 to 4478942
Data columns (total 27 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   Date_x                   datetime64[ns]
 1   home_team                object        
 2   away_team                object        
 3   Home_Score               int64         
 4   Away_Score               int64         
 5   Tournament               object        
 6   City                     object        
 7   Country                  object        
 8   Neutral                  bool          
 9   year                     int64         
 10  month                    int64         
 11  rank                     float64       
 12  Country_Full             object        
 13  Country_Abrv             object        
 14  Total_Points             float64       
 15  previous_points          float64       
 16  Rank_Change              float64       
 17  Cur_year_Average         fl

In [20]:
Combined_Data.describe

<bound method NDFrame.describe of             Date_x         home_team  ... Confederation  Date_y
0       1872-11-30          Scotland  ...           NaN     NaT
1       1873-03-08           England  ...           NaN     NaT
2       1874-03-07          Scotland  ...           NaN     NaT
3       1875-03-06           England  ...           NaN     NaT
4       1876-03-04          Scotland  ...           NaN     NaT
...            ...               ...  ...           ...     ...
4478938 2019-07-18    American Samoa  ...           NaN     NaT
4478939 2019-07-18              Fiji  ...           NaN     NaT
4478940 2019-07-19           Senegal  ...           NaN     NaT
4478941 2019-07-19        Tajikistan  ...           NaN     NaT
4478942 2019-07-20  Papua New Guinea  ...           NaN     NaT

[4478943 rows x 27 columns]>

In [21]:
Combined_Data


,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,Confederation,Date_y
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4478938,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478939,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478940,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4478941,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [37]:
# We select the Home Team details
Home_Team_Combined = Combined_Data[Combined_Data.home_team == Combined_Data.Country_Full]
Home_Team_Combined.head()



,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,Confederation,Date_y
18192,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,Barbados,BRB,0.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,1994-04-19
18291,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,29.0,Ghana,GHA,0.0,41.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18479,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,48.0,Mali,MLI,0.0,31.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18747,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,147.0,Mauritania,MTN,0.0,3.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18838,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,False,1994,4,69.0,Thailand,THA,0.0,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFC,1994-04-19


In [38]:
# We select the Away Team details
Away_Team_Combined = Combined_Data[Combined_Data.away_team == Combined_Data.Country_Full]
Away_Team_Combined.head()

,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,Confederation,Date_y
18224,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,131.0,Grenada,GRN,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,1994-04-19
18286,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,24.0,Egypt,EGY,0.0,44.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18532,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,101.0,Burkina Faso,BFA,0.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18648,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,48.0,Mali,MLI,0.0,31.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19
18774,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,False,1994,4,5.0,Nigeria,NGA,0.0,49.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-04-19


In [40]:
#we dont need all the columns so we select a subset of only what we need

Home_Team_Combined.columns
Away_Team_Combined.columns

Index(['Date_x', 'home_team', 'away_team', 'Home_Score', 'Away_Score',
       'Tournament', 'City', 'Country', 'Neutral', 'year', 'month', 'rank',
       'Country_Full', 'Country_Abrv', 'Total_Points', 'previous_points',
       'Rank_Change', 'Cur_year_Average', 'Cur_Year_Avg_Weighted',
       'Last_Year_Avg', 'Last_Year_Avg_Weighted', 'Level_of_Education',
       'Two_Year_Ago_Weighted', 'Three_Year_Ago_Avg',
       'Three_Year_Ago_Weighted', 'Confederation', 'Date_y'],
      dtype='object')

In [42]:
# Selecting only the relevant columns in the away_combined dataframe
# Creating a list of relevant columns
#
Combined_Data_Cleaned = list(Away_Team_Combined.columns[9:-2])
Combined_Data_Cleaned.extend(['home_team', 'away_team'])

Away_Team_Combined = Away_Team_Combined[Combined_Data_Cleaned]
Away_Team_Combined

,year,month,rank,Country_Full,Country_Abrv,Total_Points,previous_points,Rank_Change,Cur_year_Average,Cur_Year_Avg_Weighted,Last_Year_Avg,Last_Year_Avg_Weighted,Level_of_Education,Two_Year_Ago_Weighted,Three_Year_Ago_Avg,Three_Year_Ago_Weighted,home_team,away_team
18224,1994,4,131.0,Grenada,GRN,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Barbados,Grenada
18286,1994,4,24.0,Egypt,EGY,0.00,44.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Ghana,Egypt
18532,1994,4,101.0,Burkina Faso,BFA,0.00,12.0,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Mali,Burkina Faso
18648,1994,4,48.0,Mali,MLI,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Mauritania,Mali
18774,1994,4,5.0,Nigeria,NGA,0.00,49.0,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Thailand,Nigeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477065,2018,6,55.0,Panama,PAN,571.17,574.0,0.0,276.92,276.92,309.00,154.50,326.30,97.89,209.29,41.86,South Korea,Panama
4477407,2018,6,186.0,Mongolia,MNG,51.00,51.0,2.0,8.50,8.50,34.00,17.00,0.00,0.00,127.50,25.50,Laos,Mongolia
4477652,2018,6,9.0,Chile,CHI,1134.66,1146.0,0.0,403.99,403.99,831.33,415.67,724.98,217.49,487.56,97.51,Mexico,Chile
4477914,2018,6,60.0,Ecuador,ECU,523.85,506.0,3.0,67.14,67.14,387.50,193.75,630.02,189.01,369.77,73.95,Oman,Ecuador


In [134]:
# We need to combine the Home Team and Away Team data frames to then proceed to answer our reserach questions
Football_Teams_Data = pd.merge(Home_Team_Combined, Away_Team_Combined, how = 'left' , left_on =['year', 'month', 'home_team', 'away_team'],\
                    right_on = ['year', 'month', 'home_team', 'away_team'])
Football_Teams_Data

,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,rank_x,Country_Full_x,Country_Abrv_x,Total_Points_x,previous_points_x,Rank_Change_x,Cur_year_Average_x,Cur_Year_Avg_Weighted_x,Last_Year_Avg_x,Last_Year_Avg_Weighted_x,Level_of_Education_x,Two_Year_Ago_Weighted_x,Three_Year_Ago_Avg_x,Three_Year_Ago_Weighted_x,Confederation,Date_y,score,Result,rank_y,Country_Full_y,Country_Abrv_y,Total_Points_y,previous_points_y,Rank_Change_y,Cur_year_Average_y,Cur_Year_Avg_Weighted_y,Last_Year_Avg_y,Last_Year_Avg_Weighted_y,Level_of_Education_y,Two_Year_Ago_Weighted_y,Three_Year_Ago_Avg_y,Three_Year_Ago_Weighted_y
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,Barbados,BRB,0.00,13.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CONCACAF,1994-04-19,0,Draw,131.0,Grenada,GRN,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,Barbados,BRB,0.00,13.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CONCACAF,1994-04-19,0,Draw,131.0,Grenada,GRN,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,29.0,Ghana,GHA,0.00,41.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1994-04-19,1,Win,24.0,Egypt,EGY,0.00,44.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,48.0,Mali,MLI,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1994-04-19,0,Draw,101.0,Burkina Faso,BFA,0.00,12.0,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,147.0,Mauritania,MTN,0.00,3.0,-1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1994-04-19,-2,Lose,48.0,Mali,MLI,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20558,2018-10-16,Laos,Mongolia,1,4,Friendly,Vientiane,Laos,False,2018,6,178.0,Laos,LAO,81.58,80.0,1.0,21.25,21.25,65.52,32.76,63.57,19.07,42.50,8.50,AFC,2018-06-07,-3,Lose,186.0,Mongolia,MNG,51.00,51.0,2.0,8.50,8.50,34.00,17.00,0.00,0.00,127.50,25.50
20559,2018-10-16,Malaysia,Kyrgyzstan,0,1,Friendly,Malacca City,Malaysia,False,2018,6,171.0,Malaysia,MAS,93.04,104.0,0.0,28.48,28.48,44.80,22.40,109.29,32.79,46.93,9.39,AFC,2018-06-07,-1,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20560,2018-10-16,Mexico,Chile,0,1,Friendly,Queretaro,Mexico,False,2018,6,15.0,Mexico,MEX,989.44,1008.0,0.0,470.55,470.55,506.78,253.39,579.97,173.99,457.54,91.51,CONCACAF,2018-06-07,-1,Lose,9.0,Chile,CHI,1134.66,1146.0,0.0,403.99,403.99,831.33,415.67,724.98,217.49,487.56,97.51
20561,2018-10-16,Oman,Ecuador,0,0,Friendly,Doha,Qatar,True,2018,6,84.0,Oman,OMA,391.06,383.0,3.0,259.44,259.44,114.64,57.32,192.68,57.80,82.45,16.49,AFC,2018-06-07,0,Draw,60.0,Ecuador,ECU,523.85,506.0,3.0,67.14,67.14,387.50,193.75,630.02,189.01,369.77,73.95


In [135]:
# We drop columns that we will not need for our analysis
drop_cols = ['Country_Full_x', 'Country_Abrv_x', 'Date_y', 'Country_Full_y',	'Country_Abrv_y']
Football_Teams_Data.drop(drop_cols, axis = 1, inplace = True)


In [136]:
Football_Teams_Data.columns


Index(['Date_x', 'home_team', 'away_team', 'Home_Score', 'Away_Score',
       'Tournament', 'City', 'Country', 'Neutral', 'year', 'month', 'rank_x',
       'Total_Points_x', 'previous_points_x', 'Rank_Change_x',
       'Cur_year_Average_x', 'Cur_Year_Avg_Weighted_x', 'Last_Year_Avg_x',
       'Last_Year_Avg_Weighted_x', 'Level_of_Education_x',
       'Two_Year_Ago_Weighted_x', 'Three_Year_Ago_Avg_x',
       'Three_Year_Ago_Weighted_x', 'Confederation', 'score', 'Result',
       'rank_y', 'Total_Points_y', 'previous_points_y', 'Rank_Change_y',
       'Cur_year_Average_y', 'Cur_Year_Avg_Weighted_y', 'Last_Year_Avg_y',
       'Last_Year_Avg_Weighted_y', 'Level_of_Education_y',
       'Two_Year_Ago_Weighted_y', 'Three_Year_Ago_Avg_y',
       'Three_Year_Ago_Weighted_y'],
      dtype='object')

In [137]:
#We rename our Columns for consitency
Football_Teams_Data = Football_Teams_Data.rename(columns={
    'rank_x' : 'Rank_Home',
    'Total_Points_x' : 'Total_Points_Home',
    'previous_points_x' :'Previous_Points_Home', 
     'Rank_Change_x':'Rank_Change_Home',
     'Cur_year_Average_x': 'Curv_year_Average_Home', 
     'Cur_Year_Avg_Weighted_x' : 'Curv_Year_Avg_Weighted_Home', 
     'Last_Year_Avg_x' : 'Last_Year_Avg_Home',
     'Last_Year_Avg_Weighted_x' : 'Last_Year_Avg_Weighted_Home', 
     'Level_of_Education_x' : 'Level_of_Education_Home',
     'Two_Year_Ago_Weighted_x' : 'Two_Year_Ago_Weighted_Home', 
     'Three_Year_Ago_Avg_x' : 'Three_Year_Ago_Avg_Home',
     'Three_Year_Ago_Weighted_x' : 'Three_Year_Ago_Weighted_Home', 
     'rank_y': 'Rank_Away',
     'Total_Points_y': 'Total_Points_Away', 
     'previous_points_y' : 'Previous_Points_Away', 
     'Rank_Change_y' : 'Rank_Change_Away',
     'Cur_year_Average_y': 'Cur_year_Average_Away', 
     'Cur_Year_Avg_Weighted_y':'Cur_Year_Avg_Weighted_Away', 
     'Last_Year_Avg_y':'Last_Year_Avg_Away',
     'Last_Year_Avg_Weighted_y': 'Last_Year_Avg_Weighted_Away',
     'Level_of_Education_y':'Level_of_Education_Away',
     'Two_Year_Ago_Weighted_y':'Two_Year_Ago_Weighted_Away', 
     'Three_Year_Ago_Avg_y':'Three_Year_Ago_Avg_Away',
     'Three_Year_Ago_Weighted_y' :'Three_Year_Ago_Weighted_Away'
})
Football_Teams_Data.columns


Index(['Date_x', 'home_team', 'away_team', 'Home_Score', 'Away_Score',
       'Tournament', 'City', 'Country', 'Neutral', 'year', 'month',
       'Rank_Home', 'Total_Points_Home', 'Previous_Points_Home',
       'Rank_Change_Home', 'Curv_year_Average_Home',
       'Curv_Year_Avg_Weighted_Home', 'Last_Year_Avg_Home',
       'Last_Year_Avg_Weighted_Home', 'Level_of_Education_Home',
       'Two_Year_Ago_Weighted_Home', 'Three_Year_Ago_Avg_Home',
       'Three_Year_Ago_Weighted_Home', 'Confederation', 'score', 'Result',
       'Rank_Away', 'Total_Points_Away', 'Previous_Points_Away',
       'Rank_Change_Away', 'Cur_year_Average_Away',
       'Cur_Year_Avg_Weighted_Away', 'Last_Year_Avg_Away',
       'Last_Year_Avg_Weighted_Away', 'Level_of_Education_Away',
       'Two_Year_Ago_Weighted_Away', 'Three_Year_Ago_Avg_Away',
       'Three_Year_Ago_Weighted_Away'],
      dtype='object')

In [138]:
#We check for missing values
Football_Teams_Data.isnull().sum()

Date_x                             0
home_team                          0
away_team                          0
Home_Score                         0
Away_Score                         0
Tournament                         0
City                               0
Country                            0
Neutral                            0
year                               0
month                              0
Rank_Home                          0
Total_Points_Home                  0
Previous_Points_Home               0
Rank_Change_Home                   0
Curv_year_Average_Home             0
Curv_Year_Avg_Weighted_Home        0
Last_Year_Avg_Home                 0
Last_Year_Avg_Weighted_Home        0
Level_of_Education_Home            0
Two_Year_Ago_Weighted_Home         0
Three_Year_Ago_Avg_Home            0
Three_Year_Ago_Weighted_Home       0
Confederation                      0
score                              0
Result                             0
Rank_Away                       1811
T

We have a large number of missing values that we will drop

In [139]:
#we drop the missing values
Football_Teams_Data.dropna(inplace=True)

In [140]:
#we confirm that our changes have been succesful
Football_Teams_Data.isnull().sum()

Date_x                          0
home_team                       0
away_team                       0
Home_Score                      0
Away_Score                      0
Tournament                      0
City                            0
Country                         0
Neutral                         0
year                            0
month                           0
Rank_Home                       0
Total_Points_Home               0
Previous_Points_Home            0
Rank_Change_Home                0
Curv_year_Average_Home          0
Curv_Year_Avg_Weighted_Home     0
Last_Year_Avg_Home              0
Last_Year_Avg_Weighted_Home     0
Level_of_Education_Home         0
Two_Year_Ago_Weighted_Home      0
Three_Year_Ago_Avg_Home         0
Three_Year_Ago_Weighted_Home    0
Confederation                   0
score                           0
Result                          0
Rank_Away                       0
Total_Points_Away               0
Previous_Points_Away            0
Rank_Change_Aw

In [141]:
#we check the data types in the merged data set
Football_Teams_Data.dtypes


Date_x                          datetime64[ns]
home_team                               object
away_team                               object
Home_Score                               int64
Away_Score                               int64
Tournament                              object
City                                    object
Country                                 object
Neutral                                   bool
year                                     int64
month                                    int64
Rank_Home                              float64
Total_Points_Home                      float64
Previous_Points_Home                   float64
Rank_Change_Home                       float64
Curv_year_Average_Home                 float64
Curv_Year_Avg_Weighted_Home            float64
Last_Year_Avg_Home                     float64
Last_Year_Avg_Weighted_Home            float64
Level_of_Education_Home                float64
Two_Year_Ago_Weighted_Home             float64
Three_Year_Ag

In [142]:
#We check for duplicated values
Football_Teams_Data.duplicated().any()

True

In [143]:
# we drop the duplicates
Football_Teams_Data.drop_duplicates(keep = 'first',inplace=True)

In [144]:
#We confirm that the duplicate values have been dropped
Football_Teams_Data.duplicated().any()

False

In [145]:
Football_Teams_Data_Backup = Football_Teams_Data
Football_Teams_Data_Backup

,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,Rank_Home,Total_Points_Home,Previous_Points_Home,Rank_Change_Home,Curv_year_Average_Home,Curv_Year_Avg_Weighted_Home,Last_Year_Avg_Home,Last_Year_Avg_Weighted_Home,Level_of_Education_Home,Two_Year_Ago_Weighted_Home,Three_Year_Ago_Avg_Home,Three_Year_Ago_Weighted_Home,Confederation,score,Result,Rank_Away,Total_Points_Away,Previous_Points_Away,Rank_Change_Away,Cur_year_Average_Away,Cur_Year_Avg_Weighted_Away,Last_Year_Avg_Away,Last_Year_Avg_Weighted_Away,Level_of_Education_Away,Two_Year_Ago_Weighted_Away,Three_Year_Ago_Avg_Away,Three_Year_Ago_Weighted_Away
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,0.00,13.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CONCACAF,0,Draw,131.0,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,29.0,0.00,41.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1,Win,24.0,0.00,44.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,0,Draw,101.0,0.00,12.0,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,147.0,0.00,3.0,-1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,-2,Lose,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,False,1994,4,69.0,0.00,25.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,AFC,0,Draw,5.0,0.00,49.0,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20557,2018-10-16,Japan,Uruguay,4,3,Friendly,Saitama,Japan,False,2018,6,61.0,520.75,528.0,-1.0,164.51,164.51,413.01,206.50,273.24,81.97,338.88,67.78,AFC,1,Win,14.0,1018.41,976.0,3.0,486.28,486.28,487.69,243.84,561.47,168.44,599.26,119.85
20558,2018-10-16,Laos,Mongolia,1,4,Friendly,Vientiane,Laos,False,2018,6,178.0,81.58,80.0,1.0,21.25,21.25,65.52,32.76,63.57,19.07,42.50,8.50,AFC,-3,Lose,186.0,51.00,51.0,2.0,8.50,8.50,34.00,17.00,0.00,0.00,127.50,25.50
20560,2018-10-16,Mexico,Chile,0,1,Friendly,Queretaro,Mexico,False,2018,6,15.0,989.44,1008.0,0.0,470.55,470.55,506.78,253.39,579.97,173.99,457.54,91.51,CONCACAF,-1,Lose,9.0,1134.66,1146.0,0.0,403.99,403.99,831.33,415.67,724.98,217.49,487.56,97.51
20561,2018-10-16,Oman,Ecuador,0,0,Friendly,Doha,Qatar,True,2018,6,84.0,391.06,383.0,3.0,259.44,259.44,114.64,57.32,192.68,57.80,82.45,16.49,AFC,0,Draw,60.0,523.85,506.0,3.0,67.14,67.14,387.50,193.75,630.02,189.01,369.77,73.95


In [146]:
Football_Teams_Data_Backup['Result'] = Football_Teams_Data_Backup.Home_Score - Football_Teams_Data_Backup.Away_Score
Football_Teams_Data_Backup.Result

0        0
2        1
3        0
4       -2
5        0
        ..
20557    1
20558   -3
20560   -1
20561    0
20562    0
Name: Result, Length: 17840, dtype: int64

In [147]:
def  Result(goals):
  if goals > 0:
    return 'Win'
  elif goals < 0:
    return 'Lose'
  else:
    return 'Draw'

#We assign our resuls to our dataframe

Football_Teams_Data_Backup['Result'] = Football_Teams_Data_Backup['Result'].apply(lambda goals: Result(goals))
Football_Teams_Data_Backup



,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,Rank_Home,Total_Points_Home,Previous_Points_Home,Rank_Change_Home,Curv_year_Average_Home,Curv_Year_Avg_Weighted_Home,Last_Year_Avg_Home,Last_Year_Avg_Weighted_Home,Level_of_Education_Home,Two_Year_Ago_Weighted_Home,Three_Year_Ago_Avg_Home,Three_Year_Ago_Weighted_Home,Confederation,score,Result,Rank_Away,Total_Points_Away,Previous_Points_Away,Rank_Change_Away,Cur_year_Average_Away,Cur_Year_Avg_Weighted_Away,Last_Year_Avg_Away,Last_Year_Avg_Weighted_Away,Level_of_Education_Away,Two_Year_Ago_Weighted_Away,Three_Year_Ago_Avg_Away,Three_Year_Ago_Weighted_Away
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,0.00,13.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CONCACAF,0,Draw,131.0,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,29.0,0.00,41.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1,Win,24.0,0.00,44.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,0,Draw,101.0,0.00,12.0,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,147.0,0.00,3.0,-1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,-2,Lose,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,False,1994,4,69.0,0.00,25.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,AFC,0,Draw,5.0,0.00,49.0,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20557,2018-10-16,Japan,Uruguay,4,3,Friendly,Saitama,Japan,False,2018,6,61.0,520.75,528.0,-1.0,164.51,164.51,413.01,206.50,273.24,81.97,338.88,67.78,AFC,1,Win,14.0,1018.41,976.0,3.0,486.28,486.28,487.69,243.84,561.47,168.44,599.26,119.85
20558,2018-10-16,Laos,Mongolia,1,4,Friendly,Vientiane,Laos,False,2018,6,178.0,81.58,80.0,1.0,21.25,21.25,65.52,32.76,63.57,19.07,42.50,8.50,AFC,-3,Lose,186.0,51.00,51.0,2.0,8.50,8.50,34.00,17.00,0.00,0.00,127.50,25.50
20560,2018-10-16,Mexico,Chile,0,1,Friendly,Queretaro,Mexico,False,2018,6,15.0,989.44,1008.0,0.0,470.55,470.55,506.78,253.39,579.97,173.99,457.54,91.51,CONCACAF,-1,Lose,9.0,1134.66,1146.0,0.0,403.99,403.99,831.33,415.67,724.98,217.49,487.56,97.51
20561,2018-10-16,Oman,Ecuador,0,0,Friendly,Doha,Qatar,True,2018,6,84.0,391.06,383.0,3.0,259.44,259.44,114.64,57.32,192.68,57.80,82.45,16.49,AFC,0,Draw,60.0,523.85,506.0,3.0,67.14,67.14,387.50,193.75,630.02,189.01,369.77,73.95


In [148]:
Football_Teams_Data_Backup 

,Date_x,home_team,away_team,Home_Score,Away_Score,Tournament,City,Country,Neutral,year,month,Rank_Home,Total_Points_Home,Previous_Points_Home,Rank_Change_Home,Curv_year_Average_Home,Curv_Year_Avg_Weighted_Home,Last_Year_Avg_Home,Last_Year_Avg_Weighted_Home,Level_of_Education_Home,Two_Year_Ago_Weighted_Home,Three_Year_Ago_Avg_Home,Three_Year_Ago_Weighted_Home,Confederation,score,Result,Rank_Away,Total_Points_Away,Previous_Points_Away,Rank_Change_Away,Cur_year_Average_Away,Cur_Year_Avg_Weighted_Away,Last_Year_Avg_Away,Last_Year_Avg_Weighted_Away,Level_of_Education_Away,Two_Year_Ago_Weighted_Away,Three_Year_Ago_Avg_Away,Three_Year_Ago_Weighted_Away
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,False,1994,4,99.0,0.00,13.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CONCACAF,0,Draw,131.0,0.00,7.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,False,1994,4,29.0,0.00,41.0,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,1,Win,24.0,0.00,44.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,False,1994,4,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,0,Draw,101.0,0.00,12.0,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,False,1994,4,147.0,0.00,3.0,-1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,CAF,-2,Lose,48.0,0.00,31.0,9.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,False,1994,4,69.0,0.00,25.0,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,AFC,0,Draw,5.0,0.00,49.0,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20557,2018-10-16,Japan,Uruguay,4,3,Friendly,Saitama,Japan,False,2018,6,61.0,520.75,528.0,-1.0,164.51,164.51,413.01,206.50,273.24,81.97,338.88,67.78,AFC,1,Win,14.0,1018.41,976.0,3.0,486.28,486.28,487.69,243.84,561.47,168.44,599.26,119.85
20558,2018-10-16,Laos,Mongolia,1,4,Friendly,Vientiane,Laos,False,2018,6,178.0,81.58,80.0,1.0,21.25,21.25,65.52,32.76,63.57,19.07,42.50,8.50,AFC,-3,Lose,186.0,51.00,51.0,2.0,8.50,8.50,34.00,17.00,0.00,0.00,127.50,25.50
20560,2018-10-16,Mexico,Chile,0,1,Friendly,Queretaro,Mexico,False,2018,6,15.0,989.44,1008.0,0.0,470.55,470.55,506.78,253.39,579.97,173.99,457.54,91.51,CONCACAF,-1,Lose,9.0,1134.66,1146.0,0.0,403.99,403.99,831.33,415.67,724.98,217.49,487.56,97.51
20561,2018-10-16,Oman,Ecuador,0,0,Friendly,Doha,Qatar,True,2018,6,84.0,391.06,383.0,3.0,259.44,259.44,114.64,57.32,192.68,57.80,82.45,16.49,AFC,0,Draw,60.0,523.85,506.0,3.0,67.14,67.14,387.50,193.75,630.02,189.01,369.77,73.95


We are now ready to do EDA. We will make a copy of our dataframe in case we need to rollback


#Exploratory Data Analysis


In [ ]:
#We check for o

#Polynomial Regression

In [150]:
#we will use a smaller subset of the Columns available and pick those
Poly_Data = Football_Teams_Data_Backup[['home_team',	'away_team',	'Home_Score',	'Away_Score',	'Tournament', 'Rank_Home', 'Rank_Away']]
Poly_Data

,home_team,away_team,Home_Score,Away_Score,Tournament,Rank_Home,Rank_Away
0,Barbados,Grenada,0,0,Friendly,99.0,131.0
2,Ghana,Egypt,2,1,Friendly,29.0,24.0
3,Mali,Burkina Faso,1,1,Friendly,48.0,101.0
4,Mauritania,Mali,1,3,Friendly,147.0,48.0
5,Thailand,Nigeria,1,1,Friendly,69.0,5.0
...,...,...,...,...,...,...,...
20557,Japan,Uruguay,4,3,Friendly,61.0,14.0
20558,Laos,Mongolia,1,4,Friendly,178.0,186.0
20560,Mexico,Chile,0,1,Friendly,15.0,9.0
20561,Oman,Ecuador,0,0,Friendly,84.0,60.0


# We create our polynomial model

In [153]:

X = Poly_Data.iloc[:, [0, 1, 3, 4, 5, 6]]
y = Poly_Data.Home_Score


X = pd.get_dummies(X, drop_first=True)

# We split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

# We then standardise the X_train and the X_test to the same scale
#
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# We fit the polynomial features to the X_train and X_test and train the model
#
poly_features = PolynomialFeatures(degree = 1)
X_train = poly_features.fit_transform(X_train)
X_test = poly_features.fit_transform(X_test)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Making predictions

y_pred = regressor.predict(X_test)

# We measure the accuracy of our model
#
print(np.sqrt(mean_squared_error(y_test, y_pred)))

16283857193.518692


We have a high RSME Score which is bad  as a lowe sxcore indicates a better probability of predicting the score